## Resultados:
Para todos, Precision 1 entre 0.73 e 0.74.

1. Random forest:   
Recall 0:  0.10

2. Random forest pos-feature selection:
Recall 0: 0.10

3. Random forest pos-feature selection and with gridsearch best_params_: {'max_features': 'log2', 'n_estimators': 100}
Recall 0:  0.10

4. Ada Boost:
Recall 0: 0.29

5. Ada Boost with gridsearch best_params_: {'learning_rate': 0.0001, 'n_estimators': 25, 'random_state': 10}
Recall 0: 0.00 

6. XGBoost
Recall 0: 0.08

7. XGBoost with gridsearch best_params_: {'booster': 'gbtree', 'gamma': 0.0001, 'learning_rate': 0.1}
Recall 0:  0.04

# Objetivo: Focar no recall do zero - minimizar false positive (dar credito para 0)

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from wanna_buy_house.data import final_treatment
from wanna_buy_house.utils import final_transformer

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier 
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
data_cleaned = final_treatment()
data = final_transformer(data_cleaned)
data

get_data 0.17


,Loan.Status,Credit.Minus.Loan,Years.current_job_enc,Tax.Liens.Enc,leverage,Bankruptcies.enc,Years.since.last.delinquent,Loan.vs.Income,Current.Loan.Amount.Scaled,Annual.Income.Scaled,...,H.O.Home Mortgage,H.O.Own Home,H.O.Rent,Purp.Business Loan,Purp.Buy House,Purp.Buy a Car,Purp.Debt Consolidation,Purp.Home Improvements,Purp.Medical Bills,Purp.Other
0,1,1,6,0,0.190000,0,1,0.187495,0.480640,0.812322,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,1,4,0,0.301998,1,0,0.207542,-0.081566,-0.177278,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,1,10,0,0.085501,0,0,0.075625,-0.720623,-0.326846,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1,0,5,0,0.127000,1,0,0.165112,-0.361279,-0.326894,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0,0,10,0,0.056798,0,0,0.075445,-0.626010,0.032061,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60794,1,0,6,0,0.047101,0,0,0.119199,0.228620,1.518039,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
60795,1,1,1,0,0.197000,0,0,0.100010,-0.596212,-0.251632,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
60797,1,1,10,0,0.292000,0,0,0.279996,1.319108,0.909699,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
60800,0,0,3,0,0.103000,0,1,0.200004,0.510101,0.711574,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
#XGBoost Model complained Year.Credit.History.Labeled was category dtype
#Converted Year.Credit.History.Labeled from category to int64

data['Year.Credit.History.Labeled'] = data['Year.Credit.History.Labeled'].astype('int64')

In [28]:
# Customized metric - recall0= TP0/(TP0+FN0); precision0= TP0/(TP0+FP0)
# F1 0 = 2 * ((precision * recall) / (precision + recall)

def custom_metric(y_true, y_pred):
    
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    
    for pred, real in zip(y_pred, y_true):
        if pred == 0:
            if real == 0: 
                tp += 1 #acertei TP
            else:
                fp += 1 #errei FP
        else:
            if real == 1: 
                tn += 1 #acertei TN
            else: 
                fn += 1 #errei FN
    
    if (tp + fn) == 0 or (tp + fp) == 0:
        return 0
    else:
        return 2 * ((tp / (tp + fn) * (tp / (tp+fp))) / ((tp / (tp + fn)) + (tp / (tp+fp))))

In [29]:
from sklearn.metrics import make_scorer

custom_metric = make_scorer(custom_metric)

In [8]:
#Baseline
29984/(29984+11192)

0.7281911793277638

In [9]:
# Defining variables
# For X, also dropped Annual.Income.Scaled and Current.Loan.Amount.Scaled --> 
# as they are embedded in the Loan.vs.Income ratio

X = data.drop(columns=['Loan.Status', 'Annual.Income.Scaled', 'Current.Loan.Amount.Scaled'])
y = data['Loan.Status'] 

# Train Test Split both datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. Random Forest Model

## 1.1 The model

In [40]:
# Random forest model
#class_weight='balanced' pela sugestao do Prof. Filipe para o Armando
forest = RandomForestClassifier(class_weight='balanced')

forest_cv = cross_validate(forest,
                          X_train,
                          y_train,
                          scoring=custom_metric,
                          cv=10)
#scoring alternativo: f1_weighted

# Model fit, score and score metrics
forest.fit(X_train, y_train)
forest.score(X_test, y_test)
y_pred_forest = forest.predict(X_test)

print("F1 score: ",metrics.f1_score(y_test, y_pred_forest, average='weighted'))
#print("Recall: ",metrics.recall_score(y_test, y_pred_forest))
print(classification_report(y_test, y_pred_forest))

F1 score:  0.6462058279378036
              precision    recall  f1-score   support

           0       0.39      0.11      0.17      3373
           1       0.74      0.94      0.82      8980

    accuracy                           0.71     12353
   macro avg       0.57      0.52      0.50     12353
weighted avg       0.64      0.71      0.65     12353



In [32]:
df = pd.DataFrame(y_pred_forest)
df.value_counts()

1    11444
0      909
dtype: int64

In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_forest)

array([[ 358, 3015],
       [ 551, 8429]], dtype=int64)

## 1.2. Feature selection

In [13]:
# Feature selection

permutation_score = permutation_importance(forest, X_train, y_train, n_repeats=1)

importance = np.vstack((X.columns, permutation_score.importances_mean)).T
importance_df = pd.DataFrame(data=importance)
importance_df.columns=['variable', 'value'] 
importance_df
df = importance_df.sort_values(by=['value'], ascending=False)
df

,variable,value
6,Loan.vs.Income,0.211914
3,leverage,0.200847
1,Years.current_job_enc,0.148701
10,Credit.Score.Labeled,0.143254
7,Term.Encoded,0.115845
11,Year.Credit.History.Labeled,0.0972487
8,Number.of.Open.Accounts.Labeled,0.0845852
0,Credit.Minus.Loan,0.0845852
14,H.O.Rent,0.0732401
5,Years.since.last.delinquent,0.068487


In [14]:
# Dropping three additional variables to generate X_fs:
# (1) Tax.Liens.Enc, (2) H.O.Own Home, (3)Bankruptcies.enc, (4) Number.of.Credit.Problems.Labeled
X_fs = data.drop(columns=['Loan.Status', 'Annual.Income.Scaled', 
                          'Current.Loan.Amount.Scaled', 'Tax.Liens.Enc', 
                          'H.O.Own Home', 'Bankruptcies.enc', 
                         'Number.of.Credit.Problems.Labeled'])
y = data['Loan.Status']

In [34]:
X_train_fs, X_test_fs, y_train, y_test = train_test_split(X_fs, y, test_size=0.3, random_state=42)

# Random forest model
forest_fs = RandomForestClassifier(class_weight='balanced', n_jobs=-1)

forest_cv_fs = cross_validate(forest_fs,
                          X_train_fs,
                          y_train,
                          scoring=custom_metric,
                          cv=10)

forest_fs.fit(X_train_fs, y_train)
forest_fs.score(X_test_fs, y_test)
y_pred_forest_fs = forest_fs.predict(X_test_fs)

#print("F1 score: ",metrics.f1_score(y_test, y_pred_forest_fs))
#print("Recall: ",metrics.recall_score(y_test, y_pred_forest_fs))
print(classification_report(y_test, y_pred_forest))

              precision    recall  f1-score   support

           0       0.39      0.11      0.17      3373
           1       0.74      0.94      0.83      8980

    accuracy                           0.71     12353
   macro avg       0.57      0.52      0.50     12353
weighted avg       0.64      0.71      0.65     12353



## 1.3. GridSearch for Random Forest

In [16]:
#GridSearch for forest_fs model

#param_grid = {'n_estimators':[100, 1000, 2000],
#               'max_features': ["auto", "sqrt", "log2"],    
#}

#CV_forest_fs = GridSearchCV(estimator=forest_fs, scoring=custom_metric, param_grid=param_grid, cv=10, verbose=1, n_jobs=-1)
#CV_forest_fs.fit(X_train_fs, y_train)
#print(CV_forest_fs.best_params_)

## 1.4. Model with best_params_

In [17]:
#Applied GridSearch (gs) --> best_params = {'log2', 'n_estimators': 100}

forest_fs_gs = RandomForestClassifier(class_weight='balanced', n_jobs=-1, 
                                   max_features='log2', n_estimators=100)

forest_cv_fs_gs = cross_validate(forest_fs_gs,
                          X_train_fs,
                          y_train,
                          scoring=custom_metric,
                          cv=10)

forest_fs_gs.fit(X_train_fs, y_train)
forest_fs_gs.score(X_test_fs, y_test)
y_pred_forest_fs_gs = forest_fs_gs.predict(X_test_fs)

#print("F1 score: ",metrics.f1_score(y_test, y_pred_forest_fs_gs))
#print("Recall: ",metrics.recall_score(y_test, y_pred_forest_fs_gs))
print(classification_report(y_test, y_pred_forest))

              precision    recall  f1-score   support

           0       0.39      0.11      0.17      3373
           1       0.74      0.94      0.82      8980

    accuracy                           0.71     12353
   macro avg       0.56      0.52      0.50     12353
weighted avg       0.64      0.71      0.65     12353



# 2. AdaBoost Model

## 2.1. The model

In [35]:
# AdaBoostClassifier model

DTC = DecisionTreeClassifier(class_weight = "balanced", splitter='random', criterion='entropy', max_features = "auto", max_depth = None)

ada_model = AdaBoostClassifier(base_estimator=DTC)

ada_model.fit(X_train, y_train)
ada_model.score(X_test, y_test)

y_pred_ada = ada_model.predict(X_test)

#print("F1 score: ",metrics.f1_score(y_test, y_pred_ada))
#print("Recall: ",metrics.recall_score(y_test, y_pred_ada))

print(classification_report(y_test, y_pred_ada))

              precision    recall  f1-score   support

           0       0.32      0.31      0.32      3373
           1       0.74      0.76      0.75      8980

    accuracy                           0.63     12353
   macro avg       0.53      0.53      0.53     12353
weighted avg       0.63      0.63      0.63     12353



In [19]:
df1= pd.DataFrame(y_pred_ada)
df1.value_counts()

1    9079
0    3274
dtype: int64

## 2.2. GridSearch for AdaBoost

In [20]:
#GridSearch for ada_model

#param_grid_ada = {'n_estimators':[25, 50, 75, 100],
#                 'learning_rate':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 1.0],
#                  'random_state':[0, 10, 20, 42],
#                 }

#CV_ada = GridSearchCV(estimator=ada_model, param_grid=param_grid_ada, 
#                      cv=10, verbose=1, n_jobs=-1)
#CV_ada.fit(X_train, y_train)
#print(CV_ada.best_params_)

## 2.3. AdaBoost model with best_params_

In [36]:
#Applied GridSearch (gs) --> best_params = {'learning_rate': 0.0001, 'n_estimators': 25, 'random_state': 10}

ada_model_gs = AdaBoostClassifier(learning_rate=0.0001, n_estimators=25, random_state=10)

ada_model_gs.fit(X_train, y_train)
ada_model_gs.score(X_test, y_test)

y_pred_ada_gs = ada_model_gs.predict(X_test)

#print("F1 score: ",metrics.f1_score(y_test, y_pred_ada_gs))
#print("Recall: ",metrics.recall_score(y_test, y_pred_ada_gs))
print(classification_report(y_test, y_pred_ada_gs))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3373
           1       0.73      1.00      0.84      8980

    accuracy                           0.73     12353
   macro avg       0.36      0.50      0.42     12353
weighted avg       0.53      0.73      0.61     12353



c:\users\m_day\.venvs\lewagon\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 3. XGBoost

## 3.1. The model

In [37]:
xgb_model = XGBClassifier()

xgb_model.fit(X_train, y_train)
xgb_model.score(X_test, y_test)

y_pred_xgb = xgb_model.predict(X_test)

print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.46      0.08      0.14      3373
           1       0.74      0.96      0.83      8980

    accuracy                           0.72     12353
   macro avg       0.60      0.52      0.49     12353
weighted avg       0.66      0.72      0.64     12353



In [39]:
df2= pd.DataFrame(y_pred_xgb)
df2.value_counts()

1    11751
0      602
dtype: int64

## 3.2. Hyperparameter tuning for XGBoost

In [23]:
#param_grid_xgb = {'learning_rate':[0.0001, 0.001, 0.01, 0.1, 0.5, 1.0],
#                  'booster':['gbtree', 'gblinear', 'dart'],
#                  'gamma': [0.0001, 0.001, 0.01, 0.1, 1.0]
#                 }

#CV_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, 
#                      cv=10, verbose=1, n_jobs=-1)
#CV_xgb.fit(X_train, y_train)
#print(CV_xgb.best_params_)

## 3.3. XGBoost optimized